In [1]:
#load_ext autotime
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

<ipython-input-1-863b54eaad45>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [2]:
locator = Nominatim(user_agent="measurements")

In [3]:
odf = pd.read_csv("T:/DCProjects/StoryMap/BikeCounting/BikeShare/Data/trips_org_dst_20_21.csv")
odf.head()

C:\Users\clid1852\.conda\envs\geoenv\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,OriginDestination
0,12920035,19645,1218271,Broadway & Willamette,44.04975833333334,-123.09266666666667,2020-01-01,00:14,Origin
1,12920121,19817,893464,Broadway & High,44.04996945423723,-123.08788329362869,2020-01-01,01:25,Origin
2,12920122,19698,1700302,Broadway & High,44.04996945423723,-123.08788329362869,2020-01-01,01:26,Origin
3,12920151,19583,761821,"13th & Pearl, Pearl side",44.04547,-123.08943,2020-01-01,02:11,Origin
4,12920154,19583,761821,Broadway & Willamette,44.049648355184594,-123.09269249439241,2020-01-01,02:16,Origin


In [25]:
odf = odf[odf.Latitude != ' - ']

In [26]:
df = odf

In [27]:
df["geom"] = df["Latitude"].map(str) + ',' + df['Longitude'].map(str)
df["geom"][0]

'44.04975833333334,-123.09266666666667'

In [28]:
locator = Nominatim(user_agent="measurements", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.00001)

In [29]:
tqdm_notebook.pandas()

In [30]:
df['address'] = df['geom'].progress_apply(rgeocode)

  0%|          | 0/354698 [00:00<?, ?it/s]

RateLimiter caught an error, retrying (0/2 tries). Called with (*('44.0576,-123.10359166666667',), **{}).
Traceback (most recent call last):
  File "C:\Users\clid1852\.conda\envs\geoenv\lib\site-packages\urllib3\connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\clid1852\.conda\envs\geoenv\lib\site-packages\urllib3\util\connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\Users\clid1852\.conda\envs\geoenv\lib\socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11002] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\clid1852\.conda\envs\geoenv\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\clid1852\.conda\envs\g

In [31]:
df.address[1]

Location(Whole Foods Market, 353, East Broadway, Eugene, Lane County, Oregon, 97401, United States, (44.0503814, -123.08773676230663, 0.0))

In [32]:
df.Location.isnull().values[3]

False

In [33]:
def getLoc(x):
    loc = x.raw['address']
    a = [key for key, value in loc.items()][0]
    b = [key for key, value in loc.items()][1]
    if  a == 'house_number':
        res = loc['road'] + ' ' + loc['house_number']
    else:
        res = loc[a]
        if res == "University of Oregon":
            res = "UO " + loc[b]
    return res

In [34]:
getLoc(df[df.Location.isnull().values].address.values[1])

'Fern Ridge Path'

In [35]:
df['Name'] = df.address.apply(lambda x: getLoc(x))

In [36]:
sorted(df.Name.unique())

['105.5 BOB FM',
 '10th Street',
 '10th and High Street Station',
 '10th and Mill Building',
 '11th and High Street station',
 '12th Avenue Sidewalk',
 '13th & Olive',
 '13th Avenue Market',
 '13th Avenue Sidewalk',
 '13th and Olive (Willamate)',
 '16 Tons Café',
 '16 Tons Taphouse & Bottleshop',
 '1960 Cocina',
 '2125 Franklin',
 '3000s - Brooks Field',
 '4000s - Brooks Field',
 '4th Street 100',
 '4th Street 104',
 '4th Street 105',
 '4th Street 108',
 '4th Street 110',
 '4th Street 112',
 '4th Street 114',
 '4th Street 241',
 '4th Street 243',
 '4th Street 245',
 '4th Street 248',
 '5000s - Brooks Field',
 '5000s/6000s Brooks Field',
 '541 Sushi Bar',
 '5th Street Public Market',
 '66 Motel',
 '6th Street Alley',
 '7-Eleven',
 '76',
 '8000s - Brooks Field',
 '959 Franklin Apartments',
 'AT&T',
 'Abandoned Car',
 'Abbie Lane',
 'Academy of Arts and Academics',
 'Accountants on the GO, LLC',
 'Acorn Park Street 1101',
 'Actors Cabaret of Eugene',
 'Adams Alley 2467',
 'Adams Elementar

for n in df.loc[df.Location.isnull(),'Name'].unique():
    if all(df.loc[df['Name'] == n, 'Location'].isnull()):
        df.loc[df['Name'] == n, 'Location'] = n
    else:
        v = df.loc[df['Name'] == n, 'Location']
        df.loc[((df.Location.isnull()) & df['Name'] == n), 'Location'] = v[v.notnull()].values[0]    

In [37]:
df['Location'] = df['Location'].str.replace('University of Oregon','UO')

In [38]:
def remove_street_number(x):
    x = str(x)
    if len(x.split(' ')[-2:-1]) == 0:
        res = x
    elif (x.split(' ')[-2:-1][0] in ['Avenue', 'Street', 'Alley']) and (x.split(' ')[-1:][0].isdigit()):
        res = x.rsplit(' ', 1)[0]
    else:
        res = x
    
    return res

In [39]:
df['Location'] = df['Location'].apply(lambda x: remove_street_number(x))

In [40]:
df['Location'] = df['Location'].str.replace(' -',',')

df.loc[df.Location == 'nan', 'Location'] = df.loc[df.Location == 'nan', 'Name']

In [41]:
for i in df.RouteID.unique():
    idx1 = ((df['RouteID'] == i) & (df['OriginDestination'] =='Origin'))
    idx2 = ((df['RouteID'] == i) & (df['OriginDestination'] =='Destination'))
    pathID = df.loc[idx1, 'Location'].values[0] + ' - ' + df.loc[idx2, 'Location'].values[0]
    df.loc[idx1, 'PathID'] = pathID
    df.loc[idx2, 'PathID'] = pathID

In [42]:
df.tail()

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,OriginDestination,geom,address,Name,PathID
354775,14209403,19712,2211448,17th & Hilyard,44.0410616666667,-123.081933333333,2021-12-31,22:12,Destination,"44.0410616666667,-123.081933333333","(Peace Lutheran, East 17th Avenue, College Hil...",Peace Lutheran,"17th & Pearl, West side - 17th & Hilyard"
354776,14209412,19763,1733742,nan,44.045285,-123.080093333333,2021-12-31,23:03,Destination,"44.045285,-123.080093333333","(Glenwood, 1340, Alder Street, Eugene, Lane Co...",Glenwood,Monroe St & Blair Blvd - nan
354777,14209416,19631,1354450,nan,44.042795,-123.064628333333,2021-12-31,23:27,Destination,"44.042795,-123.064628333333","(Maude Kerns Art Center, Orchard Alley, Eugene...",Maude Kerns Art Center,13th & Kincaid - nan
354778,14209417,19755,1354450,nan,44.0427433333333,-123.064661666667,2021-12-31,23:27,Destination,"44.0427433333333,-123.064661666667","(1544, Villard Street, Eugene, Lane County, Or...",Villard Street 1544,13th & Kincaid - nan
354779,14209418,19628,1354450,nan,44.042865,-123.064646666667,2021-12-31,23:28,Destination,"44.042865,-123.064646666667","(1532, Villard Street, Eugene, Lane County, Or...",Villard Street 1532,13th & Kincaid - nan


In [43]:
ndf = df.drop(['geom', 'address'], axis=1)

In [44]:
ndf.tail()

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,OriginDestination,Name,PathID
354775,14209403,19712,2211448,17th & Hilyard,44.0410616666667,-123.081933333333,2021-12-31,22:12,Destination,Peace Lutheran,"17th & Pearl, West side - 17th & Hilyard"
354776,14209412,19763,1733742,nan,44.045285,-123.080093333333,2021-12-31,23:03,Destination,Glenwood,Monroe St & Blair Blvd - nan
354777,14209416,19631,1354450,nan,44.042795,-123.064628333333,2021-12-31,23:27,Destination,Maude Kerns Art Center,13th & Kincaid - nan
354778,14209417,19755,1354450,nan,44.0427433333333,-123.064661666667,2021-12-31,23:27,Destination,Villard Street 1544,13th & Kincaid - nan
354779,14209418,19628,1354450,nan,44.042865,-123.064646666667,2021-12-31,23:28,Destination,Villard Street 1532,13th & Kincaid - nan


In [45]:
ndf['Name'] = ndf['Name'].str.replace('University of Oregon','UO')
ndf['Name'] = ndf['Name'].str.replace('U of O','UO')

In [47]:
ndf.to_csv(r'T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\pathID_20_21.csv', index=False)